In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from data_process import rolling_split
import matplotlib.pyplot as plt
import seaborn

return_test_day = [1,3,5]
prediction_parameters_dic = {"Forecast period":1, "time_rolling_window":22}


df_Macro = pd.read_csv("./Data/EURUSD_Macro.csv")
df_Macro["Dates"] = pd.to_datetime(df_Macro["Dates"])
df_Macro["Last_Return"] =((df_Macro["PX_LAST"].pct_change(periods=prediction_parameters_dic["Forecast period"]))
                          )

df_Macro["Predict_Return"] = ((df_Macro["PX_LAST"].pct_change(periods=prediction_parameters_dic["Forecast period"])
                      .dropna(ignore_index = True)))

df_Macro.dropna(inplace=True)
df_Macro

,Dates,PX_OPEN,PX_HIGH,PX_LOW,PX_LAST,DXY Curncy,GOLDS Comdty,USGG2YR Index,USGG5YR Index,USGG10YR Index,...,CAC Index,DAX Index,EURUSD 1M 25DC,EURUSD 1M 25DP,EURUSDV1M ATM,EURUSDV1W ATM,EUR1M Curncy,EUR12M Curncy,Last_Return,Predict_Return
1,2003-01-03,1.0358,1.0437,1.0342,1.0428,102.470,351.25,1.7499,2.9682,4.0169,...,3187.88,3092.94,10.07,10.07,10.0000,10.1500,-13.80,-122.00,0.006369,0.002685
2,2003-01-06,1.0420,1.0491,1.0416,1.0456,101.970,351.25,1.7983,3.0239,4.0518,...,3210.27,3157.25,10.40,10.39,10.3250,9.7000,-14.25,-120.50,0.002685,-0.004495
3,2003-01-07,1.0450,1.0456,1.0401,1.0409,102.570,347.25,1.7418,2.9751,4.0053,...,3160.99,3112.77,9.87,9.87,9.8000,9.3000,-13.75,-120.80,-0.004495,0.008646
4,2003-01-08,1.0405,1.0534,1.0363,1.0499,101.870,353.63,1.7257,2.9995,4.0169,...,3094.09,2993.00,9.92,9.92,9.8500,9.6250,-14.20,-120.65,0.008646,-0.001333
5,2003-01-09,1.0500,1.0538,1.0463,1.0485,101.940,352.75,1.8552,3.1747,4.1790,...,3152.29,3037.68,9.79,9.79,9.7250,9.9500,-13.84,-119.30,-0.001333,0.008679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5598,2024-06-18,1.0734,1.0761,1.0710,1.0740,105.256,2329.46,4.7120,4.2431,4.2227,...,7628.80,18131.97,6.68,8.20,7.2575,5.4950,16.09,188.07,0.000559,0.000372
5599,2024-06-19,1.0740,1.0753,1.0725,1.0744,105.254,2328.16,4.7120,4.2431,4.2227,...,7570.20,18067.91,6.50,7.97,7.0575,5.1975,15.62,188.63,0.000372,-0.003909
5600,2024-06-20,1.0745,1.0749,1.0702,1.0702,105.588,2360.09,4.7372,4.2746,4.2594,...,7671.34,18254.18,6.32,7.75,6.8575,5.2975,15.10,188.94,-0.003909,-0.000841
5601,2024-06-21,1.0702,1.0721,1.0671,1.0693,105.796,2321.98,4.7322,4.2742,4.2554,...,7628.57,18163.52,6.27,7.66,6.8200,5.2600,15.23,190.69,-0.000841,0.003741


In [6]:
df_tech = pd.read_csv("./Data/EURUSD_OHLC.csv")
df_tech["Dates"]=pd.to_datetime(df_tech['Dates'])
df_tech = df_tech[df_tech["Dates"] <= pd.to_datetime("2024-06-25") ] 

df_tech["Last_Return"] =((df_tech["PX_LAST"].pct_change(periods=prediction_parameters_dic["Forecast period"]))
                          )

df_tech["Predict_Return"] = ((df_tech["PX_LAST"].pct_change(periods=prediction_parameters_dic["Forecast period"])
                      .dropna(ignore_index = True)))

df_tech.dropna(inplace=True)


df_tech

,Dates,PX_OPEN,PX_HIGH,PX_LOW,PX_LAST,Last_Return,Predict_Return
1,1980-01-03,1.5177,1.5177,1.5177,1.5177,0.001848,-0.003163
2,1980-01-04,1.5129,1.5129,1.5129,1.5129,-0.003163,0.002578
3,1980-01-07,1.5168,1.5168,1.5168,1.5168,0.002578,-0.003956
4,1980-01-08,1.5108,1.5108,1.5108,1.5108,-0.003956,0.003508
5,1980-01-09,1.5161,1.5161,1.5161,1.5161,0.003508,-0.001847
...,...,...,...,...,...,...,...
11447,2024-06-18,1.0734,1.0761,1.0710,1.0740,0.000559,0.000372
11448,2024-06-19,1.0740,1.0753,1.0725,1.0744,0.000372,-0.003909
11449,2024-06-20,1.0745,1.0749,1.0702,1.0702,-0.003909,-0.000841
11450,2024-06-21,1.0702,1.0721,1.0671,1.0693,-0.000841,0.003741


In [7]:
import sys
import os
sys.path.append("./factors")
import tech_indicators

EURUSD_close = df_tech['PX_LAST'].to_numpy()
EURUSD_typical = ((df_tech['PX_HIGH']+df_tech['PX_LOW']+df_tech['PX_LAST'])/3).to_numpy()

tech_dict = {"SMA":[10],"EMA":[50],"RSI":[10,50,100],"MACD":[26],"ROC":[2],"Bollinger_Bands_lower":[20,50],"Bollinger_Bands_upper":[20,50],
              "CCI":[20,100]}

for key,value in tech_dict.items():
    for value in value:
    
        df_column_name = key+str(value)
        
        if key[:3] == "CCI":
            df_tech[df_column_name] = tech_indicators.techindicator_to_array(EURUSD_typical,getattr(tech_indicators,key),period=value)
            
        else:
            df_tech[df_column_name] = tech_indicators.techindicator_to_array(EURUSD_close,getattr(tech_indicators,key),period=value)
        
df_tech.dropna(inplace=True,ignore_index=True)


df_tech

,Dates,PX_OPEN,PX_HIGH,PX_LOW,PX_LAST,Last_Return,Predict_Return,SMA10,EMA50,RSI10,RSI50,RSI100,MACD26,ROC2,Bollinger_Bands_lower20,Bollinger_Bands_lower50,Bollinger_Bands_upper20,Bollinger_Bands_upper50,CCI20,CCI100
0,1980-05-27,1.4725,1.4725,1.4725,1.4725,0.008354,-0.003328,1.45190,1.429335,68.750000,53.746414,51.536492,0.009267,0.008354,1.431259,1.340592,1.464671,1.485016,284.727864,34.043608
1,1980-05-28,1.4676,1.4676,1.4676,1.4676,-0.003328,-0.000886,1.45396,1.429185,66.304348,54.759544,51.528245,0.011650,-0.003328,1.430308,1.339875,1.467772,1.487073,183.562457,27.108459
2,1980-05-29,1.4663,1.4663,1.4663,1.4663,-0.000886,-0.001159,1.45564,1.426798,73.564882,58.879844,52.064292,0.012238,-0.000886,1.430049,1.339321,1.470171,1.489403,142.398522,26.016046
3,1980-05-30,1.4646,1.4646,1.4646,1.4646,-0.001159,-0.001024,1.45715,1.428254,74.718275,58.635640,52.419483,0.009106,-0.001159,1.430514,1.339691,1.471906,1.491877,109.547574,24.332424
4,1980-06-02,1.4631,1.4631,1.4631,1.4631,-0.001024,0.000410,1.45876,1.429972,77.293462,60.078821,52.796671,0.007851,-0.001024,1.431261,1.340232,1.473169,1.494128,86.049131,22.846588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11347,2024-06-18,1.0734,1.0761,1.0710,1.0740,0.000559,0.000372,1.07789,1.078226,38.181818,52.954240,46.033955,-0.001772,0.000559,1.069795,1.061241,1.092745,1.092395,-110.795455,-67.966065
11348,2024-06-19,1.0740,1.0753,1.0725,1.0744,0.000372,-0.003909,1.07664,1.077846,34.824281,51.670254,45.068052,-0.003186,0.000372,1.069031,1.061244,1.092719,1.092396,-91.733650,-62.640375
11349,2024-06-20,1.0745,1.0749,1.0702,1.0702,-0.003909,-0.000841,1.07476,1.076423,39.578794,53.214101,45.830397,-0.003442,-0.003909,1.067593,1.061130,1.093027,1.092414,-105.761354,-85.628493
11350,2024-06-21,1.0702,1.0721,1.0671,1.0693,-0.000841,0.003741,1.07368,1.075887,42.947203,53.604747,46.504664,-0.003489,-0.000841,1.066134,1.061488,1.092946,1.092256,-116.238059,-107.213253


In [8]:
split_point = int(df_Macro.shape[0]*0.9)
df_train_Mac = df_Macro.iloc[:split_point,:]
df_test_Mac = df_Macro.iloc[split_point:,:]

x_train_Mac = df_train_Mac.drop(columns=["Dates","Predict_Return"]).to_numpy()
y_train_Mac = df_train_Mac.loc[:, "Predict_Return"].to_numpy()

x_test_Mac = df_test_Mac.drop(columns=["Dates","Predict_Return"]).to_numpy()
y_test_Mac = df_test_Mac.loc[:, "Predict_Return"].to_numpy()

scale_x_Mac = StandardScaler()

x_train_norm_Mac = scale_x_Mac.fit_transform(x_train_Mac)
x_test_norm_Mac = scale_x_Mac.transform(x_test_Mac)

time_delta = prediction_parameters_dic["time_rolling_window"]
x_train_norm_rolling_Mac,x_test_norm_rolling_Mac = (rolling_split(x_train_norm_Mac,time_delta)
                                                    ,rolling_split(x_test_norm_Mac,time_delta))
y_train_rolling_Mac,y_test_rolling_Mac = y_train_Mac[time_delta-1:,...],y_test_Mac[time_delta-1:,...]

threshold = 0.002

In [9]:
split_point = int(df_tech.shape[0]*0.9)
df_train_tech = df_tech.iloc[:split_point,:]
df_test_tech = df_tech.iloc[split_point:,:]

x_train_tech = df_train_tech.drop(columns=["Dates","Predict_Return"]).to_numpy()
y_train_tech = df_train_tech.loc[:, "Predict_Return"].to_numpy()

x_test_tech = df_test_tech.drop(columns=["Dates","Predict_Return"]).to_numpy()
y_test_tech = df_test_tech.loc[:, "Predict_Return"].to_numpy()

scale_x_tech = StandardScaler()

x_train_norm_tech = scale_x_tech.fit_transform(x_train_tech)
x_test_norm_tech = scale_x_tech.transform(x_test_tech)

time_delta = prediction_parameters_dic["time_rolling_window"]
x_train_norm_rolling_tech,x_test_norm_rolling_tech = (rolling_split(x_train_norm_tech,time_delta)
                                                    ,rolling_split(x_test_norm_tech,time_delta))
y_train_rolling_tech,y_test_rolling_tech = y_train_tech[time_delta-1:,...],y_test_tech[time_delta-1:,...]

threshold = 0.002

In [10]:
from data_process import labelize

y_train_rolling_label_Mac , y_test_rolling_label_Mac  =( 
                        labelize(y_train_rolling_Mac,threshold),labelize(y_test_rolling_Mac,threshold) )

test_set_Mac = torch.utils.data.TensorDataset(torch.from_numpy(x_test_norm_rolling_Mac),
                                              torch.from_numpy(y_test_rolling_label_Mac).to(torch.int64))


y_train_rolling_label_tech , y_test_rolling_label_tech  =( 
                        labelize(y_train_rolling_tech,threshold),labelize(y_test_rolling_tech,threshold) )

test_set_tech = torch.utils.data.TensorDataset(torch.from_numpy(x_test_norm_rolling_tech),
                                              torch.from_numpy(y_test_rolling_label_tech).to(torch.int64))

mix_test_num = min([len(test_set_Mac),len(test_set_tech)])
test_set_Mac = test_set_Mac[-mix_test_num:]
test_set_tech = test_set_tech[-mix_test_num:]



In [11]:
import time_net

hyperparas_Mac = {'input_dim':x_train_Mac.shape[-1],'hidden_dim':32,'hidden_nums':5,'output_dim':3,'block_layer_nums':2, 'LSTM_layer_nums':1
        , 'dropout_rate':0}
model_Mac = time_net.LSTM_Net(hyperparas=hyperparas_Mac)
model_Mac.to(dtype=torch.float64)

hyperparas_tech = {'input_dim':x_train_tech.shape[-1],'hidden_dim':32,'hidden_nums':5,'output_dim':3,'block_layer_nums':2, 'LSTM_layer_nums':1
        , 'dropout_rate':0}
model_tech = time_net.LSTM_Net(hyperparas=hyperparas_tech)
model_tech.to(dtype=torch.float64)

model_Mac.load_state_dict(torch.load("./trained_model/Mac_LSTM.pth"))
model_tech.load_state_dict(torch.load("./trained_model/Tec_LSTM.pth"))

<All keys matched successfully>

In [19]:
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from data_process import vote

Mac_result = F.softmax(model_Mac(test_set_Mac[0]),dim=-1).detach().numpy()
tech_result = F.softmax(model_tech(test_set_tech[0]),dim=-1).detach().numpy()

mix_result = vote(Mac_result,tech_result)


real_result = test_set_tech[1].detach().numpy()

accuracy_score(mix_result,real_result)



0.40370370370370373